## 데이터 정제

1 차 정제. 형변환, buget / revenue 0 인 행들 삭제. genre explode

In [38]:
import pandas as pd
import ast

# === 1) CSV 로드 ===
df = pd.read_csv("./data_processed/tmdb_kr_theatrical_2005_2025.csv")

print("원본 shape:", df.shape)
display(df.head())
df.info()

# === 2) budget/revenue == 0 → 행 제거 ===
df = df[(df["budget"] != 0) & (df["revenue"] != 0)]

# === 3) ROI 계산 ===
df["roi"] = (df["revenue"] - df["budget"]) / df["budget"]

# === 4) release_date 날짜형 변환 ===
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")

# === 5) release_year / release_month → int (nullable) 변환 ===
df["release_year"]  = df["release_year"].astype("Int64")
df["release_month"] = df["release_month"].astype("Int64")

# === 6) 필수 컬럼 결측 제거 ===
df = df.dropna(subset=["budget", "revenue", "release_date", "genres"])

# === 7) genres 문자열 → 리스트 파싱 ===
def parse_genres(x):
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    s = str(x).strip()
    # JSON/리스트 문자열이면 literal_eval
    if s.startswith("[") and s.endswith("]"):
        try:
            val = ast.literal_eval(s)
            # 리스트 안에 dict가 있을 수도 있음: {"name": "..."}
            out = []
            for item in val:
                if isinstance(item, dict):
                    out.append(item.get("name"))
                else:
                    out.append(str(item).strip())
            return [g for g in out if g]
        except Exception:
            pass
    # 혹시 "드라마, 로맨스"처럼 쉼표 구분일 수도 있음
    if "," in s:
        return [t.strip() for t in s.split(",") if t.strip()]
    # 단일 장르 문자열인 경우
    return [s] if s else []

df["genres"] = df["genres"].apply(parse_genres)

# === 8) 장르가 비어있는 행 제거 ===
df = df[df["genres"].apply(lambda lst: isinstance(lst, list) and len(lst) > 0)]

# === 9) 장르 explode ===
df_exploded = df.explode("genres", ignore_index=True)

# 빈 문자열/결측 방지
df_exploded = df_exploded[df_exploded["genres"].notna() & (df_exploded["genres"].str.strip() != "")]

# === 10) 저장 ===
df.to_csv("./data_processed/tmdb_kr_theatrical_clean.csv", index=False, encoding="utf-8-sig")
df_exploded.to_csv("./data_processed/tmdb_kr_theatrical_clean_exploded.csv", index=False, encoding="utf-8-sig")

# === 11) 확인 ===
print("explode 후 shape:", df_exploded.shape)
display(df_exploded[["title","genres","release_year","revenue","roi"]].head(10))
df_exploded.info()

원본 shape: (4662, 16)


,movie_id,title,original_title,original_language,release_date,runtime,budget,revenue,vote_average,vote_count,popularity,genres,production_companies,production_countries,release_year,release_month
0,670,올드보이,올드보이,ko,2003-11-21,120,3000000,17500000,8.252,9150,11.6819,"['드라마', '스릴러', '미스터리', '액션']","['Show East', 'Egg Film', 'Cineclick Asia']",['KR'],2003.0,11.0
1,1255,괴물,괴물,ko,2006-07-27,119,11000000,88489643,6.980,2931,4.4383,"['공포', '드라마', 'SF']","['Chungeorahm Film', 'Showbox']",['KR'],2006.0,7.0
2,1963,활,활,ko,2005-05-12,88,950000,2032404,6.921,208,1.2098,['드라마'],"['Kim Ki Duk Film', 'Happinet Pictures', 'Cine...","['JP', 'KR']",2005.0,5.0
3,2015,밀양,밀양,ko,2007-05-23,142,2900000,11581469,7.193,215,1.6151,['드라마'],"['Cinema Service', 'Pinehouse Film', 'CJ Enter...",['KR'],2007.0,5.0
4,4550,친절한 금자씨,친절한 금자씨,ko,2005-07-29,112,0,23803308,7.500,1752,3.3480,"['드라마', '스릴러']","['TSJ Entertainment', 'Ilshin Capital Investme...",['KR'],2005.0,7.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4662 entries, 0 to 4661
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movie_id              4662 non-null   int64  
 1   title                 4662 non-null   object 
 2   original_title        4662 non-null   object 
 3   original_language     4662 non-null   object 
 4   release_date          4660 non-null   object 
 5   runtime               4662 non-null   int64  
 6   budget                4662 non-null   int64  
 7   revenue               4662 non-null   int64  
 8   vote_average          4662 non-null   float64
 9   vote_count            4662 non-null   int64  
 10  popularity            4662 non-null   float64
 11  genres                4662 non-null   object 
 12  production_companies  4662 non-null   object 
 13  production_countries  4662 non-null   object 
 14  release_year          4660 non-null   float64
 15  release_month        

,title,genres,release_year,revenue,roi
0,올드보이,드라마,2003,17500000,4.833333
1,올드보이,스릴러,2003,17500000,4.833333
2,올드보이,미스터리,2003,17500000,4.833333
3,올드보이,액션,2003,17500000,4.833333
4,괴물,공포,2006,88489643,7.044513
5,괴물,드라마,2006,88489643,7.044513
6,괴물,SF,2006,88489643,7.044513
7,활,드라마,2005,2032404,1.139373
8,밀양,드라마,2007,11581469,2.993610
9,디 워,판타지,2007,75108998,1.347156


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   movie_id              251 non-null    int64         
 1   title                 251 non-null    object        
 2   original_title        251 non-null    object        
 3   original_language     251 non-null    object        
 4   release_date          251 non-null    datetime64[ns]
 5   runtime               251 non-null    int64         
 6   budget                251 non-null    int64         
 7   revenue               251 non-null    int64         
 8   vote_average          251 non-null    float64       
 9   vote_count            251 non-null    int64         
 10  popularity            251 non-null    float64       
 11  genres                251 non-null    object        
 12  production_companies  251 non-null    object        
 13  production_countries

In [1]:
# kobis_clean_and_explode.py
import pandas as pd
import ast
from datetime import datetime

# === 0) 경로 ===
PATH_IN  = "./data_processed/kobis_weekly_2005_2025_merged.csv"
PATH_OUT = "./data_processed/kobis_weekly_clean.csv"
PATH_OUT_EXP = "./data_processed/kobis_weekly_clean_exploded.csv"

# === 1) 로드 ===
# movie_id는 문자열 유지
df = pd.read_csv(PATH_IN, dtype={"movie_id": str})

print("원본 shape:", df.shape)
display(df.head())
df.info()

# === 2) 스키마 정리: 없을 수 있는 컬럼 보강 ===
for col in ["budget", "vote_average", "vote_count", "popularity"]:
    if col not in df.columns:
        df[col] = pd.NA

# 숫자형 캐스팅 (있는 경우만)
num_cols = ["runtime", "revenue", "audience_total", "vote_average", "vote_count", "popularity"]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# 날짜형 캐스팅
if "release_date" in df.columns:
    df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")

# 년/월 보완(비어있으면 release_date에서 채움)
if "release_year" not in df.columns:
    df["release_year"] = pd.NA
if "release_month" not in df.columns:
    df["release_month"] = pd.NA

mask_missing_year = df["release_year"].isna() & df["release_date"].notna()
df.loc[mask_missing_year, "release_year"] = df.loc[mask_missing_year, "release_date"].dt.year

mask_missing_month = df["release_month"].isna() & df["release_date"].notna()
df.loc[mask_missing_month, "release_month"] = df.loc[mask_missing_month, "release_date"].dt.month

df["release_year"]  = pd.to_numeric(df["release_year"], errors="coerce").astype("Int64")
df["release_month"] = pd.to_numeric(df["release_month"], errors="coerce").astype("Int64")

# === 3) 필터 ===
#  - budget은 전부 결측일 수 있으므로 조건에서 제외
#  - revenue>0, release_date 존재, 장르 존재만 확인
if "revenue" in df.columns:
    df = df[df["revenue"].fillna(0) > 0]
if "release_date" in df.columns:
    df = df[df["release_date"].notna()]

# === 4) genres 파싱 ===
def parse_genres(x):
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    s = str(x).strip()
    # JSON/파이썬 리스트 문자열 -> 리스트
    if s.startswith("[") and s.endswith("]"):
        try:
            val = ast.literal_eval(s)
            out = []
            for item in val:
                if isinstance(item, dict):
                    out.append(item.get("name") or item.get("genreNm") or str(item))
                else:
                    out.append(str(item).strip())
            return [g for g in out if g]
        except Exception:
            pass
    # "드라마, 로맨스" 형태
    if "," in s:
        return [t.strip() for t in s.split(",") if t.strip()]
    # 단일 문자열
    return [s] if s else []

if "genres" not in df.columns:
    df["genres"] = "[]"
df["genres"] = df["genres"].apply(parse_genres)

# 장르 비어있는 행 제거
df = df[df["genres"].apply(lambda lst: isinstance(lst, list) and len(lst) > 0)]

# === 5) explode ===
df_exploded = df.explode("genres", ignore_index=True)
df_exploded = df_exploded[df_exploded["genres"].notna() & (df_exploded["genres"].astype(str).str.strip() != "")]

# === 6) 저장 ===
df.to_csv(PATH_OUT, index=False, encoding="utf-8-sig")
df_exploded.to_csv(PATH_OUT_EXP, index=False, encoding="utf-8-sig")

# === 7) 확인 ===
print("정제 후 shape:", df.shape, " / explode 후 shape:", df_exploded.shape)
display(df_exploded[["title","genres","release_year","revenue","audience_total"]].head(10))
df_exploded.info()


원본 shape: (1253, 17)


,movie_id,title,original_title,original_language,release_date,runtime,budget,revenue,vote_average,vote_count,popularity,genres,production_companies,production_countries,release_year,release_month,audience_total
0,20040786,공공의 적 2,Another Public Enemy,ko,2005-01-27,149.0,NaN,2.002252e+10,NaN,NaN,NaN,"[""드라마""]","[""(주)시네마서비스"", ""(주)시네마서비스"", ""(주)시네마서비스"", ""씨제이엔터...","[""KR""]",2005,1,3123600.0
1,20040785,말아톤,Marathon,ko,2005-01-27,117.0,NaN,2.603461e+10,NaN,NaN,NaN,"[""드라마""]","[""시네라인투(주)"", ""(주)쇼박스"", ""(주)쇼박스"", ""소빅창업투자(주)"", ...","[""KR""]",2005,1,4155296.0
2,20040774,몽정기2,Wet Dreams 2,ko,2005-01-14,101.0,NaN,5.044425e+09,NaN,NaN,NaN,"[""코미디""]","[""(주)엠케이픽처스 "", ""롯데쇼핑㈜롯데엔터테인먼트"", ""롯데쇼핑㈜롯데엔터테인먼트...","[""KR""]",2005,1,802712.0
3,20040775,키다리 아저씨,Daddy-Long-Legs,ko,2005-01-14,98.0,NaN,2.766454e+09,NaN,NaN,NaN,"[""드라마""]","[""웰메이드엔터테인먼트"", ""(주)유빈픽쳐스"", ""씨제이엔터테인먼트"", ""씨제이엔터...","[""KR""]",2005,1,436973.0
4,20050008,B형 남자친구,My Boyfriend Is Type-B,ko,2005-02-03,96.0,NaN,7.005258e+09,NaN,NaN,NaN,"[""코미디""]","[""(주)시네마제니스 "", ""롯데쇼핑㈜롯데엔터테인먼트"", ""롯데쇼핑㈜롯데엔터테인먼트...","[""KR""]",2005,2,1121897.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movie_id              1253 non-null   object 
 1   title                 1253 non-null   object 
 2   original_title        1253 non-null   object 
 3   original_language     1253 non-null   object 
 4   release_date          1253 non-null   object 
 5   runtime               1250 non-null   float64
 6   budget                0 non-null      float64
 7   revenue               1253 non-null   float64
 8   vote_average          0 non-null      float64
 9   vote_count            0 non-null      float64
 10  popularity            0 non-null      float64
 11  genres                1253 non-null   object 
 12  production_companies  1253 non-null   object 
 13  production_countries  1253 non-null   object 
 14  release_year          1253 non-null   int64  
 15  release_month        

,title,genres,release_year,revenue,audience_total
0,공공의 적 2,드라마,2005,2.002252e+10,3123600.0
1,말아톤,드라마,2005,2.603461e+10,4155296.0
2,몽정기2,코미디,2005,5.044425e+09,802712.0
3,키다리 아저씨,드라마,2005,2.766454e+09,436973.0
4,B형 남자친구,코미디,2005,7.005258e+09,1121897.0
5,그때 그 사람들,미스터리,2005,5.624003e+09,866750.0
6,그때 그 사람들,코미디,2005,5.624003e+09,866750.0
7,레드 아이,공포(호러),2005,1.092776e+09,175625.0
8,"제니, 주노",드라마,2005,1.121322e+09,189323.0
9,파송송 계란탁,코미디,2005,6.232846e+09,993352.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2027 entries, 0 to 2026
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   movie_id              2027 non-null   object        
 1   title                 2027 non-null   object        
 2   original_title        2027 non-null   object        
 3   original_language     2027 non-null   object        
 4   release_date          2027 non-null   datetime64[ns]
 5   runtime               2023 non-null   float64       
 6   budget                0 non-null      float64       
 7   revenue               2027 non-null   float64       
 8   vote_average          0 non-null      float64       
 9   vote_count            0 non-null      float64       
 10  popularity            0 non-null      float64       
 11  genres                2027 non-null   object        
 12  production_companies  2027 non-null   object        
 13  production_countri